In [ ]:
!py -m pip install pandas --user
!py -m pip install plotly --user
!py -m pip install ipykernel --user
!py -m pip install nbformat --user
!py -m pip install ipyleaflet --user
!py -m pip install geopandas --user

In [1]:
import math
import os, json
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import branca.colormap as cm
from ipyleaflet import Map, basemaps, LayersControl, LegendControl, ScaleControl, FullScreenControl, WidgetControl, Choropleth
from ipywidgets import widgets, IntSlider, link, FloatSlider, Label, HTML, Layout, interactive

Import of Regional and Global temperature readings from Kaggle and NASA.

In [2]:
data = pd.read_csv("data\Environment_Temperature_change_E_All_Data_NOFLAG.csv",encoding='cp1252')
nasa_data = pd.read_csv("data\global_monthly_mean.csv")

Data scrubbing of empty cells and null values in Keggle sourced data. Then cleaning of unused columns.

In [ ]:
print("Number of NaN values before filling :\n", data.isnull().sum())

numeric_columns = data.select_dtypes(include=['number']).columns
data.fillna(data[numeric_columns].mean(), inplace=True)

print("\nNumber of NaN values after filling :\n", data.isnull().sum())

data.columns = data.columns.str.replace("Y","")
data = data.loc[data['Element'] == 'Temperature change']
data.drop(['Area Code', 'Months Code', 'Element Code', 'Element', 'Unit'], axis=1, inplace=True)
data = pd.melt(data, id_vars=['Area', 'Months'], var_name='Year', value_name='Change in °C')


NASA's GISS Surface Temperature Analysis (v4) Monthly Global-mean.

In [4]:
years = nasa_data["Year"].tolist()
monthly_column_data = nasa_data.iloc[:,0:12]

nasa_data_melted = pd.melt(monthly_column_data, id_vars=["Year"], var_name="Month", value_name="Change in °C")

fig = px.line(nasa_data_melted, x="Year", y="Change in °C", color="Month")
fig.show()


Now that we've seen Nasa's monthly change in temperature, we want to now use the average change in temperature for the each year to compare it to the Keggle data's country specific data.

In [ ]:
# Setting up our Keggle data to pull the entire yearly mean of temperature change.
queried_data = data[data["Months"] == "Meteorological year"]
queried_data.drop("Months", axis=1 , inplace=True)
print("Area yearly change in C:\n", queried_data)

# Doing the same with Nasa's data.
yearly_mean_data = nasa_data[["Year","J-D"]]
yearly_mean_data_trimmed = yearly_mean_data[(yearly_mean_data["Year"] > 1960) & (yearly_mean_data["Year"] < 2020)]
yearly_mean_data_trimmed.rename(columns={'J-D':"Global Change in °C"}, inplace=True)

# Setting the global data into a dictionary for use on the map later.
print("global yearly change in C:\n",yearly_mean_data_trimmed)
yearly_mean_data_trimmed_dict = yearly_mean_data_trimmed.set_index("Year").to_dict()
yearly_mean_data_trimmed_dict = yearly_mean_data_trimmed_dict.get("Global Change in °C")


Now we combine the datasets, and get the combined data in a format for ipyleaflet's Choropleth to use.

In [ ]:
# Country boundaries shapefile data
selDf = gpd.read_file('_e_110m_admin_0_countries.zip')
geoDf = selDf.to_crs(4326)
areas = geoDf["SOVEREIGNT"].tolist()

# Preprocessing queried_data
# Ensure 'Year' and 'Global Change in °C' are of the correct data type
queried_data['Year'] = queried_data['Year'].astype(int)
queried_data["Change in °C"] = queried_data["Change in °C"].astype(float)

# Merging yearly_mean_data_trimmed with queried_data
merged_data = queried_data.merge(yearly_mean_data_trimmed, on="Year")
merged_data["Global Change in °C"] = merged_data["Global Change in °C"].astype(float)

# Initialize dictionary to store results
geo_total_years = {}

# Processing of all countries' data
for year in range(1961, 2019):
    geo_temps_dict = {}

    for area in areas:
        area_query = merged_data[merged_data["Area"] == area]
        
        if area_query.empty:
            geo_temps_dict[area] = math.nan
        else:
            value = area_query.loc[area_query["Year"] == year, "Change in °C"].values - area_query.loc[area_query["Year"] == year, "Global Change in °C"].values 
            geo_temps_dict[area] = value[0] if value.size > 0 else math.nan

    geo_total_years[year] = geo_temps_dict

    

Now we create the Choropleth with Ipyleaflet.

In [7]:
# Function to update hoverover value for UI.
def update_html(feature, **kwargs):
    data_value = layer.choro_data.get(feature['properties']['SOVEREIGNT'], 'No data')
    if isinstance(data_value, float):
        data_value = f"{data_value:.2f}"
    country_label.value = f"{feature['properties']['SOVEREIGNT']}: {data_value}°C"

# Function to update Choropleth layer.
def update_layer(change):
    new_year = change['new']
    new_data = geo_total_years.get(new_year, {})
    layer.choro_data = new_data
    global_label.value = f"Global Mean Change: {yearly_mean_data_trimmed_dict.get(new_year)}°C"

m = Map(center=(0, 0), zoom=2, height=5)

lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean()/2
latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean()/2

geo_json = json.loads(geoDf.set_index(geoDf["SOVEREIGNT"]).to_json())
initial_year = 1961
choro_data_dict = geo_total_years.get(initial_year)

layer = Choropleth(geo_data=geo_json,
    choro_data=choro_data_dict,
    colormap=cm.LinearColormap(['blue', 'yellow', 'red'], vmin=3., vmax=10.),
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

legend = LegendControl({"Below Global Mean": "blue", "Equal to Global Mean": "yellow", "Higher than Global Mean": "red"}, 
                       title="Mean Change in °C", 
                       position="bottomright")
                       
# Initilization of Label Widgets to show Global change in temp and focused Countries change in temp.
global_label = Label()
global_label.value = f"Global Mean Change : {yearly_mean_data_trimmed_dict.get(initial_year)}°C"
country_label = Label()
global_widget_control = WidgetControl(widget=global_label, position='topright')
country_widget_control = WidgetControl(widget=country_label, position='topright')
layer.on_hover(update_html)

# Slider widget
slider = IntSlider(value=initial_year, min=min(geo_total_years.keys()), max=max(geo_total_years.keys()), step=1, description='Year:')

# Link slider to the update function
slider.observe(update_layer, 'value')

# Add slider to the map
widget_control_slider = WidgetControl(widget=slider, position='topright')
m.add_control(widget_control_slider)

m.center = (latCent,lonCent)
m.add_layer(layer)
m.add_control(legend)
m.add_control(global_widget_control)
m.add_control(country_widget_control)

m

Map(center=[19.124565088270266, 20.412004561499185], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Sources:

GISTEMP Team, 2023: GISS Surface Temperature Analysis (GISTEMP), version 4. NASA Goddard Institute for Space Studies. Dataset accessed 20YY-MM-DD at https://data.giss.nasa.gov/gistemp/.

Lenssen, N., G. Schmidt, J. Hansen, M. Menne, A. Persin, R. Ruedy, and D. Zyss, 2019: Improvements in the GISTEMP uncertainty model. J. Geophys. Res. Atmos., 124, no. 12, 6307-6326, doi:10.1029/2018JD029522.

Country geojson data provided by Natural Earth. Free vector and raster map data @ naturalearthdata.com.